# HAI Deepfake - Ultimate Inference Pipeline

**목표**: 로컬에서 전처리(RetinaFace Crop)를 완료한 데이터를 사용하여, **초고속 & 고정확도** 추론을 수행합니다.

**준비물 (Google Drive):**
1. `processed_test_data.zip`: 로컬에서 전처리가 완료된 테스트 데이터셋
2. `checkpoints/best_model.pt`: (선택) 학습된 모델 가중치. 없으면 순정 모델 사용.

In [ ]:
# 1. 환경 설정 및 필수 라이브러리 설치
!nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

print("Install Dependencies...")
!pip install -q timm facenet-pytorch albumentations

In [ ]:
import os
import shutil

# 2. 데이터 준비 (드라이브 -> 로컬 복사 및 압축 해제)
# 드라이브 경로 설정 (본인 환경에 맞게 수정 필요)
DRIVE_ZIP_PATH = '/content/drive/MyDrive/HAI_Deepfake/processed_test_data.zip'
LOCAL_DATA_DIR = '/content/processed_test_data'

if not os.path.exists(LOCAL_DATA_DIR):
    print("📂 데이터 복사 및 압축 해제 중... (약간의 시간이 소요됩니다)")
    !cp {DRIVE_ZIP_PATH} /content/data.zip
    !unzip -q /content/data.zip -d /content/
    print("✅ 데이터 준비 완료!")
else:
    print("✅ 데이터가 이미 존재합니다.")

# 모델 경로 확인
MODEL_PATH = '/content/drive/MyDrive/HAI_Deepfake/checkpoints/best_model.pt'
if os.path.exists(MODEL_PATH):
    print(f"✅ 학습 모델 발견: {MODEL_PATH}")
else:
    print("⚠️ 학습 모델이 없습니다. 순정 모델(Pure Pretrained) 모드로 동작합니다.")

In [ ]:
# 3. 추론 코드 실행 (Inference Engine)
import torch
import timm
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
from timm.data import resolve_data_config, create_transform

# 설정
MODEL_NAME = "tf_efficientnetv2_m.in21k"
OUTPUT_CSV = "submission.csv"

class ProcessedDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.items = sorted(list(self.data_dir.glob('*')))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item_path = self.items[idx]
        pixel_values_list = []
        
        if item_path.is_dir():
            img_paths = sorted(list(item_path.glob('*.jpg')))
            for p in img_paths:
                img = Image.open(p).convert("RGB")
                if self.transform:
                    pixel_values_list.append(self.transform(img))
        else:
            img = Image.open(item_path).convert("RGB")
            if self.transform:
                pixel_values_list.append(self.transform(img))

        if not pixel_values_list:
            return torch.zeros(1, 3, 384, 384), item_path.name

        return torch.stack(pixel_values_list), item_path.name

def run_inference():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # 모델 로드
    print(f"Loading model: {MODEL_NAME}")
    model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=2)
    
    if os.path.exists(MODEL_PATH):
        checkpoint = torch.load(MODEL_PATH, map_location='cpu')
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']
            # 키 매핑 (DDP 등으로 인해 model. 접두사가 붙은 경우 제거)
            new_state_dict = {k.replace('model.model.', 'model.'): v for k, v in state_dict.items()} # 구조에 따라 다를 수 있음
            # 더 안전한 매핑: timm 모델은 self.model이 아닐 수 있음. 체크포인트 구조 확인 필요.
            # 여기서는 timm create_model로 만든 객체에 바로 로드 시도
            try:
                model.load_state_dict(checkpoint['model_state_dict'])
            except RuntimeError:
                # 만약 DeepfakeDetector 래퍼로 저장된 경우
                print("Load failed directly. Trying to adapt keys...")
                new_keys = {k.replace('model.', ''): v for k, v in checkpoint['model_state_dict'].items()}
                model.load_state_dict(new_keys, strict=False)
                
        print(f"✅ Loaded weights from: {MODEL_PATH}")
    else:
        print("⚠️ No checkpoint found. Running in Pure Pretrained mode.")
    
    model = model.to(device).eval()

    # Transform
    data_config = resolve_data_config(model.default_cfg, model=model)
    transform = create_transform(**data_config)

    # DataLoader
    dataset = ProcessedDataset(LOCAL_DATA_DIR, transform=transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

    # 추론
    results = {}
    print(f"🚀 Starting inference on {len(dataset)} items...")
    
    with torch.no_grad():
        for pixel_values, filenames in tqdm(dataloader):
            pixel_values = pixel_values.squeeze(0).to(device)
            filename = filenames[0]
            
            # AMP 적용
            with torch.cuda.amp.autocast():
                logits = model(pixel_values)
                probs = F.softmax(logits, dim=1)[:, 1] # Fake 확률
                
            final_prob = float(probs.mean().cpu().item())
            results[filename] = final_prob

    return results

results = run_inference()

In [ ]:
# 4. 결과 저장 및 내보내기
import pandas as pd

submission = pd.DataFrame(list(results.items()), columns=['filename', 'prob'])

# [선택] 라벨 반전이 필요한 경우 (0.5 이하 점수가 많이 나올 때 시도)
# submission['prob'] = 1.0 - submission['prob']

submission.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Submission saved: {OUTPUT_CSV}")

# 드라이브로 복사 (백업)
BACKUP_PATH = '/content/drive/MyDrive/HAI_Deepfake/submission_final.csv'
!cp {OUTPUT_CSV} {BACKUP_PATH}
print(f"💾 Drive Backup: {BACKUP_PATH}")

# 상위 10개 확인
submission.head(10)